# Fine-tuning the ASR model

Bu bölümde, Common Voice 13 veri kümesinde konuşma tanıma için Whisper'a ince ayar yapma konusunda adım adım bir kılavuz ele alacağız. Modelin 'küçük' sürümünü ve nispeten hafif bir veri kümesini kullanacağız, bu da Google Colab ücretsiz katmanında sağlanan 16GB T4 GPU gibi düşük disk alanı gereksinimleri olan herhangi bir 16GB + GPU'da oldukça hızlı bir şekilde ince ayar yapmanızı sağlar.

Daha küçük bir GPU'ya sahipseniz veya eğitim sırasında bellek sorunlarıyla karşılaşırsanız, bellek kullanımını azaltmak için verilen önerileri uygulayabilirsiniz. Tersine, daha büyük bir GPU'ya erişiminiz varsa, veriminizi en üst düzeye çıkarmak için eğitim argümanlarını değiştirebilirsiniz. Dolayısıyla, bu kılavuz GPU özelliklerinizden bağımsız olarak erişilebilirdir!

Aynı şekilde, bu kılavuz Dhivehi dili için Whisper modeline nasıl ince ayar yapılacağını özetlemektedir. Ancak burada anlatılan adımlar, Ortak Ses veri kümesindeki herhangi bir dile ve daha genel olarak Hugging Face Hub'daki herhangi bir ASR veri kümesine genelleştirilebilir. İstediğiniz bir dile hızlıca geçmek ve ana dilinizde bir Fısıltı modeline ince ayar yapmak için kodu değiştirebilirsiniz 🌍

Tamam! Şimdi bunu aradan çıkardığımıza göre, başlayalım ve ince ayar hattımızı başlatalım!

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()
common_voice["train"] = load_dataset(
    "mozilla-foundation/common_voice_13_0", "dv", split="train+validation",
    trust_remote_code=True
)

common_voice["test"] = load_dataset(
    "mozilla-foundation/common_voice_13_0", "dv", split="test",
    trust_remote_code=True
)

common_voice

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2677it [00:00, 73917.08it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 2227it [00:00, 20927.03it/s]


Generating test split: 0 examples [00:00, ? examples/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 16395it [00:00, 98800.74it/s][A


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1653it [00:00, 75315.68it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 4904
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 2212
    })
})

In [3]:
common_voice = common_voice.select_columns(["audio", "sentence"])

## Feature Extractor, Tokenizer and Processor

ASR pipeline üç aşamaya ayrılabilir:

- Ham ses girişlerini log-mel spektrogramlarına ön işleme tabi tutan özellik çıkarıcı
- Diziden diziye eşlemeyi gerçekleştiren model
- Tahmin edilen tokenleri sonradan işleyerek metne dönüştüren tokenizer

Transformers'da, Whisper modelinin sırasıyla WhisperFeatureExtractor ve WhisperTokenizer olarak adlandırılan ilişkili bir özellik çıkarıcısı ve belirteci vardır. Hayatımızı kolaylaştırmak için, bu iki nesne WhisperProcessor adı verilen tek bir sınıf altında toplanmıştır. Hem ses ön işleme hem de metin belirteci son işleme işlemlerini gerçekleştirmek için WhisperProcessor'ı çağırabiliriz. Bunu yaparken, eğitim sırasında yalnızca iki nesneyi takip etmemiz gerekir: işleyici ve model.

Çok dilli ince ayar yaparken, işlemciyi örneklendirirken "dil" ve "görev" ayarlarını yapmamız gerekir. "Dil" kaynak ses diline, görev ise konuşma tanıma için "transcribe" veya konuşma çevirisi için "translate" olarak ayarlanmalıdır. Bu argümanlar tokenizer'ın davranışını değiştirir ve hedef etiketlerin düzgün kodlandığından emin olmak için doğru şekilde ayarlanmalıdır.

Dil listesini içe aktararak Whisper tarafından desteklenen tüm olası dilleri görebiliriz:

In [4]:
from transformers.models.whisper.tokenization_whisper import TO_LANGUAGE_CODE

TO_LANGUAGE_CODE

{'english': 'en',
 'chinese': 'zh',
 'german': 'de',
 'spanish': 'es',
 'russian': 'ru',
 'korean': 'ko',
 'french': 'fr',
 'japanese': 'ja',
 'portuguese': 'pt',
 'turkish': 'tr',
 'polish': 'pl',
 'catalan': 'ca',
 'dutch': 'nl',
 'arabic': 'ar',
 'swedish': 'sv',
 'italian': 'it',
 'indonesian': 'id',
 'hindi': 'hi',
 'finnish': 'fi',
 'vietnamese': 'vi',
 'hebrew': 'he',
 'ukrainian': 'uk',
 'greek': 'el',
 'malay': 'ms',
 'czech': 'cs',
 'romanian': 'ro',
 'danish': 'da',
 'hungarian': 'hu',
 'tamil': 'ta',
 'norwegian': 'no',
 'thai': 'th',
 'urdu': 'ur',
 'croatian': 'hr',
 'bulgarian': 'bg',
 'lithuanian': 'lt',
 'latin': 'la',
 'maori': 'mi',
 'malayalam': 'ml',
 'welsh': 'cy',
 'slovak': 'sk',
 'telugu': 'te',
 'persian': 'fa',
 'latvian': 'lv',
 'bengali': 'bn',
 'serbian': 'sr',
 'azerbaijani': 'az',
 'slovenian': 'sl',
 'kannada': 'kn',
 'estonian': 'et',
 'macedonian': 'mk',
 'breton': 'br',
 'basque': 'eu',
 'icelandic': 'is',
 'armenian': 'hy',
 'nepali': 'ne',
 'mongol

Bu listede gezinirseniz, birçok dilin mevcut olduğunu, ancak Dhivehi'nin mevcut olmayan birkaç dilden biri olduğunu fark edeceksiniz! Bu, Whisper'ın Dhivehi üzerinde önceden eğitilmediği anlamına gelir. Ancak bu, Whisper üzerinde ince ayar yapamayacağımız anlamına gelmez. Bunu yaparken, Whisper'a önceden eğitilmiş kontrol noktasının desteklemediği yeni bir dil öğreteceğiz. Bu oldukça havalı, değil mi!

Yeni bir dilde ince ayar yaptığınızda, Whisper önceden eğitildiği diğer 96 dil hakkındaki bilgisinden yararlanma konusunda iyi bir iş çıkarır. Büyük ölçüde konuşursak, tüm modern diller Whisper'ın zaten bildiği 96 dilden en az birine dilsel olarak benzer olacaktır, bu nedenle bu diller arası bilgi temsili paradigmasına gireceğiz.

Whisper'a yeni bir dilde ince ayar yapmak için yapmamız gereken şey, Whisper'ın önceden eğitildiği en benzer dili bulmaktır. Dhivehi için Wikipedia makalesinde Dhivehi'nin Sri Lanka'nın Sinhalese dili ile yakından ilişkili olduğu belirtilmektedir. Dil kodlarını tekrar kontrol edersek, Sinhalese dilinin Whisper dil setinde mevcut olduğunu görebiliriz, bu nedenle dil argümanımızı güvenle "sinhalese" olarak ayarlayabiliriz.

Tamamdır! İşlemcimizi önceden eğitilmiş kontrol noktasından yükleyeceğiz, dili "sinhalese" olarak ve görevi yukarıda açıklandığı gibi "transcribe" olarak ayarlayacağız:

In [5]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language="sinhalese", task="transcribe"
)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Çoğu durumda, ince ayar yapmak istediğiniz dilin ön eğitim dilleri kümesinde olduğunu göreceksiniz, bu durumda dili doğrudan kaynak ses diliniz olarak ayarlayabilirsiniz! Dil ("English") ve görev ("transcribe") için yalnızca bir seçeneğin bulunduğu yalnızca İngilizce ince ayar için bu iki argümanın da atlanması gerektiğini unutmayın.

## Pre-Process the Data

Veri kümesi özelliklerine bir göz atalım. "Ses" sütununa özellikle dikkat edin - bu, ses girişlerimizin örnekleme oranını detaylandırır:

In [6]:
common_voice["train"].features

{'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

Giriş sesimiz 48kHz'de örneklendiğinden, Whisper özellik çıkarıcısına aktarmadan önce 16kHz'ye düşürmemiz gerekir; 16kHz, Whisper modeli tarafından beklenen örnekleme hızıdır.

Dataset'in cast_column yöntemini kullanarak ses girişlerini doğru örnekleme hızına ayarlayacağız. Bu işlem sesi yerinde değiştirmez, bunun yerine veri kümelerine yüklendiklerinde ses örneklerini anında yeniden örneklemeleri için sinyal verir:

In [7]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=sampling_rate))

Şimdi verilerimizi modele hazır hale getirmek için bir fonksiyon yazabiliriz:

- sample["audio"] çağrısı yaparak ses verilerini örnek bazında yükler ve yeniden örneklendiririz. Yukarıda açıklandığı gibi, Datasets gerekli yeniden örnekleme işlemlerini anında gerçekleştirir.
- Log-mel spektrogram giriş özelliklerini 1 boyutlu ses dizimizden hesaplamak için özellik çıkarıcıyı kullanıyoruz.
- Transkripsiyonları tokenizer kullanarak etiket kimliklerine kodluyoruz.

In [8]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["sentence"],
    )
    
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

In [9]:
common_voice = common_voice.map(
    prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1
)

Map:   0%|          | 0/4904 [00:00<?, ? examples/s]

2024-08-15 04:56:27.137746: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 04:56:27.138068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 04:56:27.306280: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Map:   0%|          | 0/2212 [00:00<?, ? examples/s]

In [10]:
max_input_length = 30.0

def is_audio_in_length_range(length):
    return length < max_input_length

common_voice["train"] = common_voice["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

Filter:   0%|          | 0/4904 [00:00<?, ? examples/s]

In [11]:
common_voice["train"]

Dataset({
    features: ['input_features', 'labels', 'input_length'],
    num_rows: 4904
})

Bu durumda aslında daha önce olduğu gibi aynı sayıda örneğe sahibiz, yani 30 saniyeden uzun örnek yok. Dil değiştirdiğinizde durum böyle olmayabilir, bu nedenle sağlamlık için bu filtre adımını yerinde tutmak en iyisidir. Bununla birlikte, verilerimiz eğitim için tamamen hazır! Devam edelim ve Whisper'a ince ayar yapmak için bu verileri nasıl kullanabileceğimize bir göz atalım.

## Training and Evaluation

Verilerimizi hazırladığımıza göre artık eğitim hattına dalmaya hazırız. Trainer bizim için ağır işlerin çoğunu yapacaktır. Tek yapmamız gereken

- Bir veri harmanlayıcı tanımlamak: veri harmanlayıcı önceden işlenmiş verilerimizi alır ve PyTorch tensörlerini model için hazırlar.

- Değerlendirme metrikleri: Değerlendirme sırasında, modeli kelime hata oranı (WER) metriğini kullanarak değerlendirmek istiyoruz. Bu hesaplamayı gerçekleştiren bir compute_metrics fonksiyonu tanımlamamız gerekiyor.

- Önceden eğitilmiş bir kontrol noktası yükleyin: önceden eğitilmiş bir kontrol noktası yüklememiz ve eğitim için doğru şekilde yapılandırmamız gerekir.

- Eğitim argümanlarını tanımlayın: bunlar Trainer tarafından eğitim programını oluştururken kullanılacaktır.

Modele ince ayar yaptıktan sonra, Dhivehi dilinde konuşmayı yazıya dökmek üzere doğru şekilde eğittiğimizi doğrulamak için test verileri üzerinde değerlendireceğiz.

### Define a Data Collator

Bir diziden diziye konuşma modeli için veri harmanlayıcı, input_features ve label'ları bağımsız olarak ele alması bakımından benzersizdir: input_features feature extractor tarafından, label'lar ise tokenizer tarafından ele alınmalıdır.

Input_features zaten 30'lara dolgulanmış ve sabit boyutlu bir log-Mel spektrogramına dönüştürülmüştür, bu nedenle tek yapmamız gereken onları toplu PyTorch tensörlerine dönüştürmektir. Bunu return_tensors=pt ile özellik çıkarıcının .pad yöntemini kullanarak yapıyoruz. Girdiler sabit boyutlu olduğu için burada ek bir dolgu uygulanmadığına dikkat edin, input_features basitçe PyTorch tensörlerine dönüştürülür.

Öte yandan, etiketler dolgusuzdur. İlk olarak tokenizer'ın .pad yöntemini kullanarak dizileri yığındaki maksimum uzunluğa kadar dolduruyoruz. Dolgu belirteçleri daha sonra -100 ile değiştirilir, böylece kayıp hesaplanırken bu belirteçler dikkate alınmaz. Daha sonra eğitim sırasında eklediğimiz transkript belirtecinin başlangıcını etiket dizisinin başından kesiyoruz.

Hem özellik çıkarıcı hem de tokenizer işlemlerini gerçekleştirmek için daha önce tanımladığımız WhisperProcessor'dan faydalanabiliriz:

In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    
    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )
        
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        
        batch["labels"] = labels
        
        return batch

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

## Evaluation Metrics

Ardından, değerlendirme setimizde kullanacağımız değerlendirme metriğini tanımlıyoruz. ASR sistemlerini değerlendirmek için 'de-facto' metrik olan Değerlendirme bölümünde tanıtılan Kelime Hata Oranı (WER) metriğini kullanacağız.

In [14]:
!pip install -q evaluate jiwer

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Daha sonra model tahminlerimizi alan ve WER metriğini döndüren bir fonksiyon tanımlamamız yeterlidir. compute_metrics adı verilen bu fonksiyon, ilk olarak label_ids'deki pad_token_id ile -100'ü değiştirir (kayıpta dolgulu belirteçleri doğru şekilde yok saymak için veri harmanlayıcıda uyguladığımız adımı geri alır). Daha sonra tahmin edilen ve etiket kimliklerini dizelere çözer. Son olarak, tahminler ve referans etiketleri arasındaki WER değerini hesaplar. Burada, noktalama işaretleri ve büyük/küçük harfler kaldırılmış 'normalleştirilmiş' transkripsiyonlar ve tahminlerle değerlendirme seçeneğimiz vardır. Transkripsiyonları normalleştirerek elde edilen WER iyileştirmesinden yararlanmak için bunu izlemenizi öneririz.

In [15]:
import evaluate
from transformers.models.whisper.english_normalizer import BasicTextNormalizer


metric = evaluate.load("wer")
normalizer = BasicTextNormalizer()

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)
    
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)
    
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]
    
    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [16]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

Gradyan kontrol noktası kullandığımız ve ikisi uyumsuz olduğu için `use_cache`'i eğitim için **False** olarak ayarlayacağız. Ayrıca çıkarım sırasında modelin davranışını kontrol etmek için iki üretim argümanını geçersiz kılacağız: dil ve görev argümanlarını ayarlayarak üretim sırasında dil ve görev belirteçlerini zorlayacağız ve ayrıca çıkarım süresini hızlandırmak için üretim için önbelleği yeniden etkinleştireceğiz:

In [17]:
from functools import partial

# gradyan kontrol noktası ile uyumsuz olduğu için eğitim sırasında önbelleği devre dışı bırakın
model.config.use_cache = False

# üretim için dili ve görevi ayarlayın ve önbelleği yeniden etkinleştirin
model.generate = partial(
    model.generate, language="sinhalese", task="transcribe", use_cache=True
)

In [18]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-dv",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=250,
    eval_steps=250,
    logging_steps=50,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

max_steps is given, it will override any value given in num_train_epochs


In [20]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer Ortho,Wer
250,0.185800,0.203400,69.691483,15.810064
500,0.074700,0.166556,60.247928,12.868171


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 65

TrainOutput(global_step=500, training_loss=0.7941111783981323, metrics={'train_runtime': 12969.1672, 'train_samples_per_second': 1.234, 'train_steps_per_second': 0.039, 'total_flos': 4.61736640512e+18, 'train_loss': 0.7941111783981323, 'epoch': 3.262642740619902})

In [21]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_13_0",
    "dataset": "Common Voice 13",
    "language": "dv",
    "model_name": "Whisper-Small-Dv-fine-tuned",
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

trainer.push_to_hub(**kwargs)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


CommitInfo(commit_url='https://huggingface.co/Leotrim/whisper-small-dv/commit/2c156254258472e3d93d70a389f2146aaf3c86f2', commit_message='End of training', commit_description='', oid='2c156254258472e3d93d70a389f2146aaf3c86f2', pr_url=None, pr_revision=None, pr_num=None)